# Cost of living

https://www.ons.gov.uk/economy/inflationandpriceindices/articles/costofliving/latestinsights   - not a useful link

https://www.ons.gov.uk/peoplepopulationandcommunity/wellbeing/articles/subnationalindicatorsexplorer/2022-01-06

https://geoportal.statistics.gov.uk/

https://jamboard.google.com/d/1UDiSgSrkWjnBMGgkC8QyGtf1ECg9ebtT3rpHshkt9yI/viewer?f=0

https://shallowsky.com/blog/mapping/folium-with-shapefiles.html#:~:text=But%20Folium%20can%27t%20handle%20shapefiles%2C%20only%20GeoJSON.%20You,do%20it%20programmatically%20with%20the%20GDAL%20Python%20bindings%3A

https://www.ons.gov.uk/economy/inflationandpriceindices/datalist?sortBy=release_date&query=regional&filter=datasets&fromDateDay=&fromDateMonth=&fromDateYear=&toDateDay=&toDateMonth=&toDateYear=

 

#!pip install geopandas
https://geoportal.statistics.gov.uk/datasets/ons::local-authority-districts-may-2023-uk-bfc-2/explore?location=54.303971%2C-0.744937%2C6.87

In [ ]:
import geopandas as gpd
import folium
import json
import geojson
import random
import branca
import pandas as pd



In [ ]:
#load as geo pandas
path_to_geojson =  'data/Local_Authority_Districts_(May_2023)_UK_BFC.geojson'
code_col = 'LAD23CD'

gj_gpd = gpd.read_file(path_to_geojson)

N = len(gj_gpd)
gj_gpd.head(3)



In [ ]:
#load as geojson
with open(path_to_geojson) as f:
    gj = geojson.load(f)


# load weight in data 

In [ ]:
#list of codes
list_codes = list(gj_gpd[code_col].drop_duplicates())
list_codes[0:10]

In [ ]:
df_weights = pd.read_csv('data/machine_readable.csv')

#rename first column
cols = list(df_weights.columns)
cols[0] = 'code'
df_weights.columns= cols

df_weights.head(2)

In [ ]:
df_weights_reduced = df_weights[df_weights['code'].isin(list_codes)]
df_weights_reduced.head(3)

In [ ]:
path = 'data/df_weight_reduced_local_authorities.csv'
df_weights_reduced.to_csv(path)

In [ ]:
#list values for uses as a weight
list_weights = list(df_weights_reduced['Unnamed: 3'].drop_duplicates())

list_weights


In [ ]:
#choose weight
weight_choice = list_weights[1]
print(weight_choice)

df_weights_reduced_single = df_weights_reduced[df_weights_reduced['Unnamed: 3'] ==weight_choice]

df_weights_reduced_single.head(2)

In [ ]:
combined1 = gj_gpd.merge(df_weights_reduced_single,left_on=code_col, right_on = 'code', how = 'inner')

combined1['Unnamed: 8'] = combined1['Unnamed: 8'].fillna(0)

In [ ]:
combined1.head(2)

In [ ]:

vmin = min(combined1['Unnamed: 8'].apply(float))
vmax =max(combined1['Unnamed: 8'].apply(float))
name = combined1.loc[0]['Unnamed: 3']
print(vmin,vmax)

colormap = branca.colormap.LinearColormap(
    vmin=vmin,
    vmax=vmax,
    colors=["red", "orange", "lightblue", "green", "darkgreen"],
    caption=f"{name}",
)


def random_html_color():
    r = random.randint(0,256)
    g = random.randint(0,256)
    b = random.randint(0,256)
    return '#%02x%02x%02x' % (r, g, b)
  

def style_fcn(x):
    return { 'fillColor': random_html_color(),
           'fillOpacity': 0.50,
           'weight' : x['properties']['weight']}


highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.01}

In [ ]:
m = folium.Map(location=[51.5072, 0.1276],zoom_start=7,zoom_control=True)
keys = list(gj[1]['properties'].keys())[0:3]
keys.append('weight')

N = len(gj_gpd)

for row in range(N):
    

    obj = gj[row]
    code = obj['properties'][code_col]
    data = combined1[combined1['code'] == code]

    if len(data)==0:
        f_weight = 0.0
    else:
        
        weight = data['Unnamed: 8']
        f_weight = float(list(weight)[0])


    gj[row]['properties']['weight'] = f_weight
    #random.randint(0,10)
    
    #folium.GeoJson(gj[row], style_function=style_fcn).add_to(m)
    feature = folium.features.GeoJson(
            obj,
            style_function=lambda x: {
            "fillColor": colormap(x["properties"]["weight"]),
                                 'fillOpacity': 0.9 
        },
           highlight_function=highlight_function,
            control=False,
            tooltip=folium.features.GeoJsonTooltip(
                fields=keys))
    
    m.add_child(feature)
    m.keep_in_front(feature)

colormap.add_to(m)
m

# Save image


In [ ]:
m.save('local_authorities_test_folium.html')